## TODO

* setup proper initalisation of model weights
* use same drop out they used in paper
* weight the underrepresented class
* k-fold validation

## things to try

* data augmentation
 

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math



In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [3]:
%matplotlib inline

In [4]:
! ls ~/data/kaggle/dont-overfit

pred-ens1.csv  sample_submission.csv  test.csv.zip
pred-ens.csv   test.csv		      train.csv


In [5]:
df_train = pd.read_csv("~/data/kaggle/dont-overfit/train.csv")

In [6]:
x = df_train.drop(columns=['target','id']).values

In [7]:
mean = np.mean(x)
std = np.std(x)

def normalize(X):
    return (X-mean)/std

In [8]:
class DofDataset(Dataset):
    def __init__(self,df, test=False):
        self.df = df
        self.test = test
            
    @property
    def tensor_labels(self):
        return torch.from_numpy(self.df.target.values).long()
    
    @property
    def tensor_data(self): #need to normalise data
        if self.test:
            return torch.from_numpy(self.df.drop(columns=['id']).values).float()
        return torch.from_numpy(self.df.drop(columns=['target', 'id']).values).float()
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        if self.test:
            return self.tensor_data[idx]
        label = self.tensor_labels[idx]
        data = self.tensor_data[idx]
        
        return label, data
        
    
        

In [9]:
# mean = np.mean(X_train)
# std = np.std(X_train)

# def standardize(x):
#     return (x-mean)/std

In [9]:
dod_ds = DofDataset(df_train)

## Metrics

In [23]:
# def accuracy(preds,labels):
#     preds_np = np.argmax(preds.detach().numpy(), axis=1)
#     return np.sum(preds_np == labels.detach().numpy())/len(preds_np)

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

## Model Defintion

In [74]:
in_size = 300
out_size = 2

model = nn.Sequential(
    nn.Linear(in_size, 600, bias=True),
    nn.SELU(),
    nn.Linear(600, 300, bias=True),
    nn.SELU(),
    nn.Dropout(p=0.7),
    nn.Linear(300, out_size, bias=True))
    


In [75]:
def init_normal(m):
    if type(m) == nn.Linear:
        size = m.in_features
        nn.init.normal_(m.weight,  mean=0, std=1/math.sqrt(size))

In [76]:
model.apply(init_normal)

Sequential(
  (0): Linear(in_features=300, out_features=600, bias=True)
  (1): SELU()
  (2): Linear(in_features=600, out_features=300, bias=True)
  (3): SELU()
  (4): Dropout(p=0.7)
  (5): Linear(in_features=300, out_features=2, bias=True)
)

## Train Definition

In [77]:
lr = 1e-4
bs = 50

cross_en = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr=lr)


In [78]:
dod_df = DataLoader(dod_ds, bs, shuffle=False)

In [79]:
def train_step(batch, train):
    labels, data = batch
    data_norm = normalize(data)
    preds = model(data_norm)
    loss = cross_en(preds, labels)
    acc = accuracy(preds,labels)
    if train:
        optim.zero_grad() # zero gradients (?? why do we do this)
        loss.backward() # calcuate gradients
        optim.step() # updated weights
    return loss.item(), acc

In [80]:
epochs = 45

In [81]:
train_loss_t = []
train_acc_t = []

In [82]:
for i in range(epochs):
    train_loss_ep = []
    train_acc_ep = []
    
    for train_batch in dod_df :
        train_loss, train_acc = train_step(train_batch, True)
        train_loss_ep.append(train_loss)
        train_acc_ep.append(train_acc)
    if i% 1 == 0:
        print(f'epoch:{i} Train Loss:  {np.mean(train_loss_ep)}, Accuracy {np.mean(train_acc_ep)}')
    train_loss_t.append(np.mean(train_loss_ep))
    train_acc_t.append(np.mean(train_acc_ep))


epoch:0 Train Loss:  1.3482912302017211, Accuracy 0.46400004625320435
epoch:1 Train Loss:  1.076285982131958, Accuracy 0.5200000405311584
epoch:2 Train Loss:  1.0576185226440429, Accuracy 0.5879999995231628
epoch:3 Train Loss:  0.9004774570465088, Accuracy 0.6000000238418579
epoch:4 Train Loss:  1.0499745607376099, Accuracy 0.5359999537467957
epoch:5 Train Loss:  0.8441011428833007, Accuracy 0.6320000290870667
epoch:6 Train Loss:  0.6997833609580993, Accuracy 0.7240000367164612
epoch:7 Train Loss:  0.7062665700912476, Accuracy 0.6839998960494995
epoch:8 Train Loss:  0.7083525776863098, Accuracy 0.6559999585151672
epoch:9 Train Loss:  0.6982383847236633, Accuracy 0.6959999799728394
epoch:10 Train Loss:  0.4911837220191956, Accuracy 0.7720000147819519
epoch:11 Train Loss:  0.5622819542884827, Accuracy 0.7440000176429749
epoch:12 Train Loss:  0.45241996049880984, Accuracy 0.7760000228881836
epoch:13 Train Loss:  0.4495764791965485, Accuracy 0.784000039100647
epoch:14 Train Loss:  0.397726

## Submission

In [64]:
df_test = pd.read_csv("~/data/kaggle/dont-overfit/test.csv")

In [65]:
test_ds = DofDataset(df_test, test=True)

In [66]:
test_dl = DataLoader(test_ds, batch_size=100)

In [67]:
preds = []

In [68]:
m = model.eval()

In [69]:
%%time
for i, batch in enumerate(test_dl):
    logits = m(batch)
    preds_np = np.argmax(logits.detach().numpy(), axis=1)
    preds.extend(preds_np)

CPU times: user 15min 12s, sys: 51.6 s, total: 16min 4s
Wall time: 15min 25s


In [70]:
df_samp = pd.read_csv("~/data/kaggle/dont-overfit/sample_submission.csv")

In [71]:
df_samp["target"] = preds

In [72]:
df_samp.to_csv("~/data/kaggle/dont-overfit/subm-snn-fixed.csv", index=False)

In [73]:
! kaggle competitions submit -c dont-overfit-ii -f ~/data/kaggle/dont-overfit/subm-snn-fixed.csv -m "fixed snn model, removed dropout from inference"

100%|████████████████████████████████████████| 144k/144k [00:07<00:00, 18.8kB/s]
Successfully submitted to Don't Overfit! II